# Detecting Couple candlestick traps by using NEAT

In [28]:
## As similar mix-1 but no hidden input

### Import Library

In [29]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns
import os

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import neat

### Load Price Data

In [31]:
import os
from pathlib import Path
notebook_path = os.getcwd()
current_dir = Path(notebook_path)
csv_file = str(current_dir.parent) + '/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print(csv_file)
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [65]:
data = dataset.copy()

In [66]:
data = data[data.index > '2020-11-01 00:00:00']

In [67]:
# len(data[(data.index > '2025-02-14 00:00:00') & (data.index < '2025-02-14 23:00:00')])

In [68]:
def set_condition_1(r):
    cond = ''
    if r['Open'] < r['Close'] <= r['High'] - 0.1:
        # Xanh va co bong nen tren
        cond = 'long'
    return cond


def set_condition_2(r):
    cond = ''
    if r['Open'] < r['Close'] == r['High'] and r['High'] > r['high_s1']:
        # Xanh va khong co bong nen tren
        cond = 'long'
    return cond
    

def get_couple_candleticks_signal(r):
    signal = ''
    if 100 * r.name.hour + r.name.minute >= 1425:
        return ''
    if r['condition_1'] == 'long' and r['condition_2'] == 'long':
        signal = 'long'
    return signal

In [69]:
# Couple candlesticks signal
data['low_s1'] = data['Low'].shift(1)
data['high_s1'] = data['High'].shift(1)
data['condition_1'] = data.apply(lambda r: set_condition_1(r), axis=1)
data['condition_1'] = data['condition_1'].shift(1)
data['condition_2'] = data.apply(lambda r: set_condition_2(r), axis=1)
data['signal'] = data.apply(lambda r: get_couple_candleticks_signal(r), axis=1)

In [70]:
data['Higher_High'] = (data['High'] > data['High'].shift(1)).astype(int)
data['Lower_Low'] = (data['Low'] < data['Low'].shift(1)).astype(int)
data['Swing_Up_Count'] = data['Higher_High'].rolling(15).sum()
data['Swing_Down_Count'] = data['Lower_Low'].rolling(15).sum()

In [71]:
len(data[data.signal != ''])

1102

In [72]:
data[data.signal != '']

,Open,High,Low,Close,Volume,low_s1,high_s1,condition_1,condition_2,signal,Higher_High,Lower_Low,Swing_Up_Count,Swing_Down_Count
Date,,,,,,,,,,,,,,
2020-11-04 10:05:00,908.2,909.0,908.1,909.0,1800,907.9,908.6,long,long,long,1,0,6.0,6.0
2020-11-04 10:15:00,910.2,911.5,909.8,911.5,3514,909.1,910.5,long,long,long,1,0,7.0,5.0
2020-11-04 10:35:00,911.2,912.5,910.8,912.5,2731,910.7,911.5,long,long,long,1,0,9.0,5.0
2020-11-04 11:00:00,913.5,914.2,912.8,914.2,1996,913.0,913.7,long,long,long,1,1,11.0,3.0
2020-11-06 13:40:00,904.5,906.7,904.4,906.7,2806,903.5,905.0,long,long,long,1,0,4.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-05 13:50:00,1331.9,1333.4,1331.5,1333.4,5276,1330.6,1331.8,long,long,long,1,0,6.0,8.0
2025-02-06 09:55:00,1342.5,1344.3,1342.3,1344.3,4723,1341.5,1342.7,long,long,long,1,0,8.0,4.0
2025-02-11 13:25:00,1331.2,1332.5,1331.2,1332.5,3534,1329.6,1331.4,long,long,long,1,0,8.0,7.0


## TRAP labeling

In [73]:
traps = []
for i, row in data.iterrows():
    if row['signal']:
        current_date = row.name.strftime('%Y-%m-%d ').format()
        current_time = row.name
        data_to_end_day = data[(data.index > current_time) & (data.index < current_date + ' 14:30:00')]
        #
        data_at_end_day = data[data.index == current_date + ' 14:25:00']
        last_close = data_at_end_day.iloc[0]['Close']
        #
        if len(data_to_end_day[data_to_end_day.Low < row['Close'] - 3.5]) > 0 or row['Close'] > last_close:
            traps.append(1)
        else:
            traps.append(0)
    else:
        traps.append('')

In [74]:
data['trap'] = traps

In [75]:
data.dropna(inplace=True)

,Open,High,Low,Close,Volume,low_s1,high_s1,condition_1,condition_2,signal,Higher_High,Lower_Low,Swing_Up_Count,Swing_Down_Count,trap
Date,,,,,,,,,,,,,,,
2020-11-02 10:10:00,899.9,899.9,898.9,899.7,1883,899.5,900.4,,,,0,1,4.0,6.0,
2020-11-02 10:15:00,899.6,901.5,899.6,901.2,2305,898.9,899.9,,,,1,0,5.0,6.0,
2020-11-02 10:20:00,901.1,901.2,899.3,899.3,1796,899.6,901.5,long,,,0,1,5.0,6.0,
2020-11-02 10:25:00,899.1,900.3,898.8,899.9,1917,899.3,901.2,,,,0,1,5.0,7.0,
2020-11-02 10:30:00,900.0,900.1,899.3,900.0,1150,898.8,900.3,long,,,0,0,5.0,6.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-14 14:15:00,1343.0,1343.0,1340.3,1341.3,7141,1342.3,1343.8,,,,0,1,3.0,9.0,
2025-02-14 14:20:00,1340.9,1341.9,1340.5,1341.4,4593,1340.3,1343.0,,,,0,0,3.0,8.0,
2025-02-14 14:25:00,1341.1,1342.5,1340.7,1342.5,4207,1340.5,1341.9,long,long,,1,0,4.0,8.0,


In [76]:
signal_data = data[data.signal != '']
signal_data.dropna(inplace=True)

In [77]:
len(signal_data)

1102

## Features

In [78]:
X = signal_data[['Swing_Up_Count', 'Swing_Down_Count', "trap"]]
# Train-Test Split
X_train = X[X.index < '2024-07-01 00:00:00']
X_test = X[X.index > '2024-07-01 00:00:00']
X_train['trap'] = X_train['trap'].astype(int)

In [79]:
len(X_train)

959

In [80]:
len(X_train[X_train.trap == 0])

354

In [81]:
X_none_trap = X_train[X_train.trap == 0]
X2_none_trap = pd.concat([X_none_trap, X_none_trap], ignore_index=True)
balanced_X_train = pd.concat([X_train, X2_none_trap], ignore_index=True)

In [82]:
len(balanced_X_train)

1667

In [83]:
len(X_test)

143

In [86]:
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        genome.fitness = 4.0
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        for move_index, row in balanced_X_train.iterrows():
            inputs = [row['Swing_Up_Count'], row['Swing_Down_Count']]
            expected_output = row['trap']
            output = net.activate(inputs)
            genome.fitness -= (output[0] - expected_output) ** 2
            if genome.fitness < -350:
                break


def run(config_file):
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    # p.add_reporter(neat.StdOutReporter(True))
    # stats = neat.StatisticsReporter()
    # p.add_reporter(stats)

    # Run for up to 100 generations.
    winner = p.run(eval_genomes, 300)

    # Display the winning genome.
    print('\nBest genome:\n{!s}'.format(winner))
    return neat.nn.FeedForwardNetwork.create(winner, config)

In [87]:
config_path = os.path.join(current_dir, 'style-swing-up-down.cfg')
best_brain = run(config_path)


Best genome:
Key: 3146
Fitness: -350.00000000000006
Nodes:
	0 DefaultNodeGene(key=0, bias=3.1744610176215478, response=1.0, activation=sigmoid, aggregation=sum)
	396 DefaultNodeGene(key=396, bias=0.5398692029302858, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 0), weight=3.8505251857709575, enabled=True)
	DefaultConnectionGene(key=(-1, 0), weight=-0.3122924525310612, enabled=False)


In [88]:
best_brain

In [89]:
# Show output of the most fit genome against training data.
outputs = []
for i, row in X_test.iterrows():
    inputs = [row['Swing_Up_Count'], row['Swing_Down_Count']]
    expected_output = row['trap']
    output = best_brain.activate(inputs)
    outputs.append(round(output[0]))
    # print("input {!r}, expected output {!r}, got {!r}".format(inputs, expected_output, output))

In [90]:
expected_outputs = X_test['trap'].to_list()
# Evaluate Performance
print("Accuracy:", accuracy_score(expected_outputs, outputs))

Accuracy: 0.6013986013986014
